# Expectation Maximization

## I. 适用场景
### I.1 有latent varible场景下的MLE问题
$$
\begin{align}
Likelihood: &\\
& L(\theta)  = \Sigma_{i  = 1}^{N}logP(x_i;\theta) \\
原目标：&\\
& \underset{\theta }{argmax}L=\underset{\theta }{argmax}\Sigma_{i  = 1}^{N}logP(x_i;\theta)\\
考虑z后的目标：&z是latent\ variable\\
1.离散分布：& \underset{\theta }{argmax}\Sigma_{i  = 1}^{N}log\Sigma_zP(x_i, z;\theta) \\
2.连续分布：& \underset{\theta }{argmax}\Sigma_{i  = 1}^{N}log\int_{z}P(x_i, z;\theta)dz 
\end{align}
$$
### I.2 有incomplete data场景下的MLE问题


## II. EM算法
### II.1 单样本分析
#### II.1.1 目标函数分析
1. 目标函数
$$logP(x_i;\theta)=log\Sigma_zP(x_i, z;\theta)$$
2. 函数性质:*[证明见附录]*
   1. 目标函数有lower bound
    $$\begin{align}
    logP(x_i;\theta)
    &=log\Sigma_zP(x_i, z;\theta) \\
    &=log\Sigma_zQ(z)*\frac{P(x_i, z;\theta)}{Q(z)}\\
    &\ge \Sigma_zQ(z)*log\frac{P(x_i, z;\theta)}{Q(z)}...[1]
    \end{align}$$<font color=red>注：$Q(z)$是一个密度函数，满足pdf的定义条件：$Q(z)>0$和$\Sigma_zQ(z)=1$。但不是z的真实pdf。</font>
   2. 当且仅当$Q(z)=P(z|x;\theta)$时，等式成立，即：$$
\begin{align}
logP(x_i;\theta) 
& = \Sigma_zQ(z)*log\frac{P(x_i, z;\theta)}{Q(z)}...[2]
\end{align}$$

#### II.1.2 Evidence lower bound (ELBO)
1. 将不等式右边记为<font color=blue>**evidence lower bound(ELBO)**</font>：
$$\begin{align}
ELBO(x;Q,\theta)& = \Sigma_zQ(z)log\frac{P(x, z;\theta)}{Q(z)}\\
logP(x_i;\theta)& \ge ELBO(x_i;Q,\theta)
\end{align}$$
2. lower bound的特征
   - ELBO给出了$logP(x_i;\theta)$的下限
   - 因为式中z会被积分积掉，所以ELBO是$x,\theta,Q$的函数。
   - 如果给定x，$P(x_i;\theta)是\theta$的函数，而ELBO是$\theta和Q$的函数。此时，如果Q取不同的分布，那么$P(x_i;\theta)$与ELBO之间的距离(差异)会变化。

#### II.1.3 优化算法
1. **思路**：<font color=blue>不断增加ELBO的大小，通过让lower bound最大化的方式来最大化原函数。</font>
   - ELBO是目标函数下限，已知x的条件下，给定$\theta$时，取$Q(z)=P(z|x;\theta)$则两者相等。那么：
     - 当$\theta=\theta_t$时，只要取$Q_t=P(z|x;\theta_t)$，则ELBO就能取到它的最大值，也就是$logP(x;\theta_t)$ 
     - 接下来Q不变，取$\theta_{t+1}=argmaxELBO(x;Q_t,\theta)$，那么改变后的ELBO比前一时刻的ELBO更大，但却比此时的$logP(x;\theta_{t+1})$小
   - 上面两种变量调整方式都会让ELBO变大，如果迭代可以收敛，即在某个时刻，$Q和\theta$都不再变化，那么此时会有$logP(x;\theta_t)=ELBO(x;Q^{*},\theta)$，$logP(x;\theta_t)$此时也会取到local min。
<img src="../pics/ELBO.png" width="560" height="360">

2. **EM算法**
    - 随机初始化$\theta_0$
    - 迭代：
      - E step(第t步)：$Q_t(z)=P(z|x;\theta_t)$ 
      - M step(第t+1步)：$\theta_{t+1}=\underset{\theta}{argmax}ELBO(x;Q_t,\theta)$

3. **算法收敛性证明：收敛到局部最优解**
   - 前面已经证明了$Q(z)=P(z|x;\theta_t)$时，$logP(x;\theta_t) 
= ELBO(x;Q,\theta_t)$，只要再证明$logP(x;\theta_t)\ge logP(x;\theta_{t+1}) $即可。
   - 证明：
$$\begin{align}
logP(x;\theta_{t+1}) & \ge ELBO(x;Q_t,\theta_{t+1}) \\
& \ge ELBO(x;Q_t,\theta_{t}) \\
& = logP(x;\theta_{t})
\end{align}$$

### II.2 多样本分析
#### II.2.1 目标函数分析
1. 目标函数
$$\Sigma_ilogP(x_i;\theta)=\Sigma_ilog\Sigma_zP(x_i, z;\theta)$$
2. 函数性质:
   1. lower bound
    $$\begin{align}
\Sigma_ilogP(x_i;\theta)
&=\Sigma_ilog\Sigma_{z_i}P(x_i, z_i;\theta) \\
&=\Sigma_ilog\Sigma_{z_i}Q(z_i)*\frac{P(x_i, z_i;\theta)}{Q(z_i)}\\
&\ge \Sigma_i\Sigma_{z_i}Q(z_i)*log\frac{P(x_i, z_i;\theta)}{Q(z_i)} \\
& = \Sigma_iELBO(x_i;Q,\theta )
\end{align}$$
   2. 当且仅当$Q(z_i)=P(z_i|x_i;\theta)$时，等式成立，即：$$
\begin{align}
\Sigma_ilogP(x_i;\theta)= \Sigma_iELBO(x_i;Q,\theta )
\end{align}$$

3. Q的特征
   - 假如有n个样本，要注意$Q(z_1),Q(z_2), ..., Q(z_n) $不是同一个Q函数的n种不同取值
   - 每个样本$(x_i,z_i)$都有自己的$Q_{z_i}(z_i)$，Q函数的类型数取决于latent varible z的特征。
   - 比如：在具体model中x来自K个分类，但每个$x_i$所属类型未知，用latent varible $z_i$表示，z的边际分布是多项式分布，即$P(z=z_k)=\phi_k, \Sigma_{k=1}^{K}\phi_k=1$。此时，Q就有K种函数形态，每种对应K种分类中的一类。

#### II.2.2 EM两步迭代
- 随机初始化$\theta_0$
- 迭代：
  - <font color=green>**E step(第t步)：$for\ i=1, 2, ..., n,取Q_{i_t}=P(z_i|x_i;\theta_{t})$。**</font>
    - 如果可以直接用分布函数，就直接代入求解。以z是离散分布为例，求解方法：  
    $$\begin{align}
Q_i(z=k) = P(z=k|x_i) & = \frac{P(x_i,z=k)}{P(x_i)}\\
& = \frac{P(x_i,z=k)}{\Sigma_{j=1}^{K}P(x_i, z=j)} \\
& = \frac{P(x_i|z=k)P(z=k)}{\Sigma_{j=1}^{K}P(x_i|z=j)P(z=j)} \\
\end{align}$$
    - 如果不能直接用分布函数，可以用sampling P(z=k|x)的方式来得到对该分布的近似估计。
  - <font color=green>**M step(第t+1步)：$$\begin{align}
\theta_{t+1}
& =\underset{\theta}{argmax}\Sigma_i ELBO(x_i;Q_{i_t},\theta) \\
& =\underset{\theta}{argmax}\Sigma_i\Sigma_{z_i}Q_i(z_i)*log\frac{P(x_i, z_i;\theta)}{Q_i(z_i)} 
\end{align}$$**</font>

### II.3 直觉上理解EM两步算法在做什么
- 如果没有缺失的信息，比如上面model中z是已知的，那么做MLE很容易。
  - 把样本中z值不同的样本分到一起，分出来K类，然后对每一类估计$P(x_i|z_k;\theta)$的参数$\theta$
  - 再用每类的样本数量来估计$P(z_k;\phi)$的参数$\phi$
- 但是有缺失信息的时候无法对样本分类，EM的intuition是：
  - E step：
    1. 用当前的参数信息来猜未知的z分布信息：P(z|x)
       - 计算方法：$P(z=k|x)=\frac{P(z=k,x)}{\Sigma_z P(z,x)}=\frac{P(x|z=k)P(z=k)}{\Sigma_z P(x|z)P(z)}$
    2. 用P(z|x)来填上缺失的信息
       - 填的方法：把样本做改造，每个样本$x_i$被分成K个子样本，每个子样本都分配了1个类型值$z=k$。样本$x_i$发生的概率1也分给这K个子样本，每个概率是$P(z=k|x_i)$
       - 原来的N个没有z信息的样本被分成了$K*N$个有z信息的样本。
  - M step：像latent varible的模型那样，用$K*N$个有z信息的样本重估参数
    - 此时，每个样本在count它的数量的时候，不再是1个子样本count1次，而是1个子样本count$P(z=k|x_i)$次。

### II.4 理解ELBO的另一种角度

<font color=blue>
$$\begin{align}
logP(x;\theta) & =ELBO(x;Q,\theta)+D_{KL}(Q(z)||P(z|x)) \\
似然 & =ELBO + KL divergence \\
等价于：\\
ELBO(x;Q,\theta) & =logP(x;\theta) - D_{KL}(Q(z)||P(z|x))
\end{align}$$
</font>
[证明见附录]

<font color=green>**变分推断中也应用了该性质，思路是：由于$logP(x;\theta)$不受$Q(z)$选择的影响，所以选择$Q(z)$让ELBO最大化相当于尽可能让$D_{KL}$逼近于0，此时$Q(z)$逼近$P(z|x)$，也就实现了对$P(z|x)$的估计。**</font>

## III. 在模型中的典型应用
### III.1 解Density Estimation for a Gaussian mixture(GMM)
#### III.1.1 GMM问题
1. 问题
   - 已知n个样本的取值，$x\in\{x_1, x_2, ..., x_n\}$，这些样本来自K种类型$z\in\{z_1, z_2, ..., z_K\}$，但每个样本属于哪种类型并不知道
   - 如果给出新的x，希望判断P(x)大小
2. 应用场景：异常检测

#### III.1.2 model
1. 模型设定：
   - 变量：
     - observed evidence: x
     - latent variable: z
   - 变量分布：
     - $z \sim multinomial(\phi)$
     - $x|z \sim N(\mu_z, \Sigma_z)$
2. 目标函数：MLE
$$\begin{align}
logL(\phi,\mu,\Sigma) & =\Sigma_{i=1}^{n}P(x_i;\phi,\mu,\Sigma) \\
& = \Sigma_{i=1}^{n}\Sigma_zP(x_i,z;\phi,\mu,\Sigma) \\
& = \Sigma_{i=1}^{n}\Sigma_{k=1}^{K}P(x_i|z=k;\mu,\Sigma)P(z=k;\phi) \\
\underset{\phi,\mu,\Sigma}{argmax}\ L &= \underset{\phi,\mu,\Sigma}{argmax} \Sigma_{i=1}^{n}\Sigma_{k=1}^{K}P(x_i|z=k;\mu,\Sigma)P(z=k;\phi)
\end{align}$$

3. 优化算法：EM迭代
   1. 初始化：$\mu_0, \Sigma_0, \phi_0$
   2. **E-step**：$for\ i=1, 2, ..., n,取Q_{i_t}=P(z_i|x_i;\theta_{t})$
         $$\begin{align}
    & Q_i(z=k) = P(z=k|x_i) = \frac{P(x_i|z=k)P(z=k)}{\Sigma_{j=1}^{K}P(x_i|z=j)P(z=j)} \\
    t时刻代入：& P(x_i|z=k) = \frac{exp\{-0.5(x_i-\mu_{tk})^T\Sigma_{tk}^{-1}(x_i-\mu_{tk})\}}{(2\pi)^{\frac{d}{2} }|\Sigma_{tk}|^{\frac{1}{2} }} \\
    &P(z=k) = \phi_{tk}
    \end{align}$$<font color=green>对每个$x_i$分别求$Q_i(z=k),k=1, 2, ..., K$
   3. **M-step**：$\theta_{t+1}=\underset{\theta}{argmax}\Sigma_i ELBO(x_i;Q_{i_t},\theta) $
        $$\begin{align}
    分别求解：&\\
    &\frac{\partial ELBO}{\partial \mu_k}=0  \rightarrow  \mu_{k,t+1}=\frac{\Sigma_{i=1}^{n}Q_i(z=k)x_i}{\Sigma_{i=1}^{n}Q_i(z=k)}\\
    &\frac{\partial ELBO}{\partial \Sigma_k}=0  \rightarrow  \Sigma_{k,t+1}=\frac{\Sigma_{i=1}^{n}Q_i(z=k)(x_i-\mu_k)(x_i-\mu_k)^T}{\Sigma_{i=1}^{n}Q_i(z=k)}\\
    &\frac{\partial ELBO}{\partial \phi_k}=0  \rightarrow  \phi_{k,t+1}=\frac{1}{n}\Sigma_{i=1}^{n}Q_i(z=k)
    \end{align}$$

4. 将优化结果代入模型处理inference问题
$$
\begin{align}
P(x) & = \Sigma_k P(x|z=k)P(z=k) \\
& = \Sigma_k N(\mu_k, \Sigma_k)\phi_k
\end{align}
$$
如果P(x)计算得到的值小于某个阈值，就可以判断x是outlier

### III.2 用EM算法解

## 附：
### I. Jensen's inequality
当f(x)为凸函数，x为随机变量时有：$$Ef(x)\ge f(Ex)$$
等式在以下情况下成立：
1. f(x)是linear function.
2. X以概率1取常数
3. 如果f(x)严格凸, 那么当且仅当X是常数时等式成立

#### 1. 证明:
<font color=blue>$$\Sigma_zQ(z)logh(z)\le log\Sigma_zQ(z)h(z)$$</font>
其中：
1. z是随机变量，有k种取值
2. Q(z)是z的概率密度函数，满足$\Sigma_zQ(z)=1$
3. h(z)与z是一对一双射，即当$z_i\ne z_j$时，$h(z_i)\ne h(z_j)$。此时，有$Q(z)=Q(h(z))$

分析：\
f(x)=log(x)时，由于log是严格凹函数，所以反过来有：$$
\begin{align}
E(log(x)) &\le log(E(x)) \\
\Sigma_{x} P(x)log(x) &\le log\Sigma_{x}P(x)x \\
\end{align}$$

因为$Q(z)=Q(h(z))$，代入上式有：
$$
\begin{align}
\Sigma_zQ(z)logh(z)
& = \Sigma_zQ(h(z))log(h(z))\\
& = \Sigma_{h(z)}Q(h(z))log(h(z))\\
& = \Sigma_HQ(H)logH \\
& \le log\Sigma_HQ(H)H \\
& = log\Sigma_{h(z)}Q(h(z))h(z) \\
& = log\Sigma_{z}Q(z)h(z) \\
\end{align}
$$

#### 2. 证明[1]式：
$$\begin{align}
logP(x_i;\theta)
&=log\Sigma_zP(x_i, z;\theta) \\
&=log\Sigma_zQ(z)*\frac{P(x_i, z;\theta)}{Q(z)}\\
&\le \Sigma_zQ(z)*log\frac{P(x_i, z;\theta)}{Q(z)}=ELBO
\end{align}$$
由Jenson不等式有：
$$\Sigma_zQ(z)logh(z)\le log\Sigma_zQ(z)h(z)$$
取函数$h(z)=\frac{P(x_i, z;\theta)}{Q(z)}$, 代入上式得：
$$
log\Sigma_zQ(z)*\frac{P(x_i, z;\theta)}{Q(z)} \le \Sigma_zQ(z)*log\frac{P(x_i, z;\theta)}{Q(z)}
$$

<font color=red>这里$Q(z)$是一个密度函数，满足pdf的定义条件：$Q(z)>0$和$\Sigma_zQ(z)=1$。但并非z的真实pdf函数。</font>

#### 3. 证明：
<font color=blue>如果f是凸函数，且h(z)是z的一对一双射，那么$$E_z[f(h(z))]\ge f(E_z[h(z)])$$ 
和Jensen不等式中一样，如果f(·)严格凸，那么当且仅当h(z)相对z是常数时上式取等。</font>\
证明：[将上一段证明方式一般化，即可得证]
$$
\begin{align}
E_z[f(h(z))]=\Sigma_zQ(z)f(h(z))\
& = \Sigma_zQ(h(z))f(h(z))\\
& = \Sigma_{h(z)}Q(h(z))f(h(z))\\
& 取H=h(z),\\
& = \Sigma_HQ(H)f(H) \\
& \ge f\left(\Sigma_HQ(H)H\right) \\
& = f(\Sigma_{h(z)}Q(h(z))h(z)) \\
& = f(\Sigma_{z}Q(z)h(z))=f(E_z[h(z)]) \\
\end{align}
$$
取等条件证明略

#### 4. 证明[2]式：
<font color=blue>当且仅当$Q(z)=P(z|x;\theta)$时下面等式成立：$$
\begin{align}
logP(x;\theta) 
& = \Sigma_zQ(z)*log\frac{P(x, z;\theta)}{Q(z)} \\
& = ELBO(x;Q,\theta)
\end{align}$$</font>
证明：\
因为log函数严格凹，所以当且仅当h(z)相对z是常数时$E_z[log(h(z))]\le log(E_z[h(z)])$取等。
$$
\begin{align}
\because h(z)& =\frac{P(x, z;\theta)}{Q(z)}=c， c是相对z的常数\\
\therefore Q(z)& =\frac{P(x, z;\theta)}{c}\\
\because \Sigma_zQ(z)&=1,\  \therefore \Sigma_z \frac{P(x, z;\theta)}{c} = \frac{\Sigma_zP(x, z;\theta )}{c}=1 \\
\therefore c&= \Sigma_zP(x, z;\theta)=P(x;\theta ) \\
\therefore 此时Q(z)& =\frac{P(x, z;\theta)}{c}=\frac{P(x, z;\theta)}{P(x;\theta )}=P(z|x;\theta )
\end{align}
$$

### II. ELBO
#### 1. 证明：似然 = ELBO + KL divergence
<font color=blue>
$$\begin{align}
logP(x;\theta) & =ELBO(x;Q,\theta)+D_{KL}(Q(z)||P(z|x)) \\
\end{align}$$
</font>
证明：
$$\begin{align}
logP(x;\theta) 
& = logP(x, z;\theta) - logP(z|x;\theta) \\
& = log\frac{P(x, z;\theta)}{Q(z)} -log\frac{P(z|x;\theta)}{Q(z)} \\
等式两边相对z的分布Q(z)求期望：\\
E_zlogP(x;\theta)
& = E_z\left (log\frac{P(x, z;\theta)}{Q(z)} -log\frac{P(z|x;\theta)}{Q(z)} \right )\\
左边& =\Sigma_zQ(z)logP(x;\theta)= logP(x;\theta)\Sigma_zQ(z)=logP(x;\theta)\\
右边
& =\Sigma_zQ(z)\left (log\frac{P(x, z;\theta)}{Q(z)} -log\frac{P(z|x;\theta)}{Q(z)} \right )\\
& = \Sigma_zQ(z)log\frac{P(x, z;\theta)}{Q(z)} -\Sigma_zQ(z)log\frac{P(z|x;\theta)}{Q(z)}\\
& = ELBO(x;Q,\theta) - D_{KL}(Q(z)||P(z|x))
\end{align}$$

<font color=blue></font>